<a href="https://colab.research.google.com/github/axm5507/spam-message-detector/blob/main/spam_message_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project involves **vectorizing** text data(converting it into a bunch of numbers for the computer to understand) and plugging the data into a Machine Learning model(Naive Bayes) to train it to recognize spam. Once done, we can test the model's accuracy and plug in our own messages to test it out.

In [ ]:
#First, we have to import all of our dependencies
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
#Now, let's read the data and clean it up/make sure it's not missing anything
message_data = pd.read_csv('message_data.csv')
message_data.columns = email_data.columns.str.strip()
message_data.isnull().sum()
message_data.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#When running head, i get the first 5 values of the dataset. The two categories are ham and spam.
#However, I'll need to change these to zeros and ones to plug into the model.
message_data['Category'] = message_data['Category'].map({'ham': 0, 'spam': 1})
#This makes all real messages zero and all fake messages one.

In [ ]:
#Now that our data is clean, lets split it into input(messages) and output(whether or not it is spam)
Y = message_data['Category']
X = message_data['Message']


In [ ]:
#Let's split our data more! We're gonna split it into training data and testing data so we can verify the accuracy of the model on the training set itself
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)


In [ ]:
#Vectorizing the text using the TF IDF model
#This model is not great in lots of scenarios because it doesn't account for context among other things, but is good as recognizing buzzwords for spam messages
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
#I'm putting the training data into the model along with the output
model = MultinomialNB()
model.fit(X_train_vectorized, Y_train)

MultinomialNB()

In [ ]:
#Now, let's check out the accuracy of the model on both the training and testing data.
#The training data accuracy should be really high b/c the model has worked with it, but the testing data accuracy shouldn't be too bad either.
train_prediction = model.predict(X_train_vectorized)
train_accuracy = accuracy_score(Y_train, train_prediction)
print(train_accuracy)

0.9807045097599282


In [ ]:
#Wow! The training data accuracy was about 98.1%. Let's check the testing data now
test_prediction = model.predict(X_test_vectorized)
test_accuracy = accuracy_score(Y_test, test_prediction)
print(test_accuracy)

0.9730941704035875


In [ ]:
#Woah. The testing data accuracy was 97.3%. It seems like our model has done a good job. Now, for good measure, lets test this out on a REAL spam message.
new_message = ["""
insert your message here
"""]

new_message_vector = vectorizer.transform(new_message)
new_prediction = model.predict(new_message_vector)
if new_prediction[0] == 1:
    print('This is a spam message')
else:
    print('This is not a spam message')
print('i am ', model.predict_proba(new_message_vector), ' sure of this')

This is not a spam message
i am  [[0.90676788 0.09323212]]  sure of this


Hmm. It seems like although this model can recognize basic spam messages with great accuracy, it struggles with advanced phishing ones. This makes sense because TF IDF does not understand language context, and phishing messages were made to sound like legitimate ones. To improve on this, I plan to use a word embeddor like spaCy that understands context, add manual flags like fraudulent link/number detectors, and simply add more realistic phishing examples to the dataset. What other next steps should I take? Let me know!
